In [1]:
# Ant Colony Optimization (ACO) for Job Scheduling Problem (Makespan Minimization)

import numpy as np

class AntColonyJobScheduling:
    def __init__(self, job_times, n_machines, n_ants, n_best, n_iterations, decay, alpha=1, beta=2):
        """
        job_times: list or array of job processing times
        n_machines: number of machines
        n_ants: number of ants per iteration
        n_best: number of best ants to deposit pheromone
        n_iterations: number of iterations
        decay: pheromone evaporation rate
        alpha: influence of pheromone
        beta: influence of heuristic (1/job_time)
        """
        self.job_times = job_times
        self.n_jobs = len(job_times)
        self.n_machines = n_machines
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

        # pheromone matrix: shape (n_jobs, n_machines)
        # pheromone[i][j] indicates desirability of assigning job i to machine j
        self.pheromone = np.ones((self.n_jobs, self.n_machines))

        # heuristic: inverse of job processing times (favor shorter jobs)
        self.heuristic = 1 / (np.array(job_times) + 1e-6)

    def run(self):
        best_solution = None
        best_makespan = float('inf')

        for _ in range(self.n_iterations):
            all_solutions = []
            for _ in range(self.n_ants):
                solution = self.construct_solution()
                makespan = self.evaluate_solution(solution)
                all_solutions.append((solution, makespan))

                if makespan < best_makespan:
                    best_solution = solution
                    best_makespan = makespan

            self.update_pheromone(all_solutions)
            self.pheromone *= (1 - self.decay)

        return best_solution, best_makespan

    def construct_solution(self):
        assignment = [-1] * self.n_jobs
        for job in range(self.n_jobs):
            probabilities = (self.pheromone[job] ** self.alpha) * (self.heuristic[job] ** self.beta)
            probabilities /= probabilities.sum()
            machine = np.random.choice(self.n_machines, p=probabilities)
            assignment[job] = machine
        return assignment

    def evaluate_solution(self, assignment):
        machine_loads = [0] * self.n_machines
        for job, machine in enumerate(assignment):
            machine_loads[machine] += self.job_times[job]
        return max(machine_loads)

    def update_pheromone(self, all_solutions):
        # sort solutions by makespan (ascending)
        all_solutions.sort(key=lambda x: x[1])
        for solution, makespan in all_solutions[:self.n_best]:
            # deposit pheromone inversely proportional to makespan
            deposit = 1.0 / makespan
            for job, machine in enumerate(solution):
                self.pheromone[job][machine] += deposit

# Example usage:
job_times = [2, 14, 4, 16, 6, 5, 3, 7, 8]
n_machines = 3

aco_scheduler = AntColonyJobScheduling(
    job_times=job_times,
    n_machines=n_machines,
    n_ants=20,
    n_best=5,
    n_iterations=100,
    decay=0.1,
    alpha=1,
    beta=2
)

best_assignment, best_makespan = aco_scheduler.run()
print(f"Best assignment: {best_assignment}")
print(f"Best makespan: {best_makespan}")


Best assignment: [0, 0, 1, 2, 0, 2, 1, 1, 1]
Best makespan: 22
